In [ ]:
from gerrychain import (GeographicPartition, Graph, 
                        updaters, Election)
import numpy as np
import statistics
import geopandas as gpd


#dont forget the histograms
#clean up the graph
#presentation most inttresting stuff (could be both)
#dont forget evidence of convertions 
#


In [31]:
def eg(part, election):
    election_results = part[election]
    dem_counts = election_results.votes("Democratic")  
    rep_counts = election_results.votes("Republican")
    
    wasted_votes_dem = 0
    wasted_votes_rep = 0

    for i in range(len(dem_counts)): # <- not super familar with python so i used chatGPT with help on how to loop through a tuple
        votes_dem = dem_counts[i]
        votes_rep = rep_counts[i]
        #calculating the total number of votes in the district and the minimum number of votes needed to win the district
        total_votes = votes_dem + votes_rep
        min_winning_votes = (total_votes // 2) + 1
        # if the democratic candidate wins the district then the wasted votes for the republican candidate is the number of votes the republican candidate received
        if votes_dem > votes_rep:
            wasted_votes_dem += votes_dem - min_winning_votes
            wasted_votes_rep += votes_rep
        else:
            wasted_votes_dem += votes_dem
            wasted_votes_rep += votes_rep - min_winning_votes
    #calculating the efficiency gap by subtracting the wasted votes of the democratic candidate from the wasted votes of the republican candidate and dividing by the total number of votes
    total_votes_all = sum(dem_counts) + sum(rep_counts)
    efficiency_gap = (wasted_votes_rep - wasted_votes_dem) / total_votes_all

    return efficiency_gap


In [32]:
def cut_edges_count(part):
    graph = part.graph
    assignment = part.assignment
    count = 0
    for node in graph.nodes:
        district = assignment[node]
        for neighbor in graph.neighbors(node):
            if assignment[neighbor] != district:
                count += 1

    return count // 2  # Each cut edge is counted twice (once from each node), so divide by 2


In [33]:
geo_df = gpd.read_file("./NM/NM.shp")
graph = Graph.from_geodataframe(geo_df)


In [34]:
my_updaters = {"population": updaters.Tally("TOTPOP", alias="population")}

elections = [
    Election("USS18", {"Democratic": "G18USSD", "Republican": "G18USSR"}),
    Election("GOV18", {"Democratic": "G18GOVD", "Republican": "G18GOVR"}),
    Election("SOS18", {"Democratic": "G18SOSD", "Republican": "G18SOSR"}),
    Election("TRE18", {"Democratic": "G18TRED", "Republican": "G18TRER"}),
    Election("ATG18", {"Democratic": "G18ATGD", "Republican": "G18ATGR"}),
    Election("AUD18", {"Democratic": "G18AUDD", "Republican": "G18AUDR"}),
    Election("LND18", {"Democratic": "G18LNDD", "Republican": "G18LNDR"}),
]


In [35]:
election_updaters = {election.name: election for election in elections}
my_updaters.update(election_updaters)


In [36]:
initial_partition = GeographicPartition(graph, 
                                        assignment= "SACD", updaters=my_updaters) 

In [37]:
ideal_population = sum(initial_partition["population"].values()) / len(initial_partition)
print(type(initial_partition[elections[0].name]))


<class 'gerrychain.updaters.election.ElectionResults'>


In [38]:
print("Partisan bias: ", initial_partition[elections[1].name].partisan_bias())


Partisan bias:  0.014285714285714235


In [39]:
print("Efficiency gap: ", eg(initial_partition, elections[1].name))

Efficiency gap:  0.07919633460117526


In [40]:
print("Cut edges:", cut_edges_count(initial_partition))


Cut edges: 1453
